In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [2]:
!pip install transformers datasets scikit-learn torch tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import hamming_loss, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
import re
from IPython.display import display

In [4]:
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/go_emotions_dataset.csv')
df = pd.read_csv('/content/go_emotions_dataset.csv')

In [5]:
df.head()

,id,text,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,eew5j0j,That game hurt.,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,eemcysk,>sexuality shouldn’t be a grouping category I...,True,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ed2mah1,"You do right, if you don't care then fuck 'em!",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,eeibobj,Man I love reddit.,False,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,eda6yn6,"[NAME] was nowhere near them, he was by the Fa...",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211225 entries, 0 to 211224
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    211225 non-null  object
 1   text                  211225 non-null  object
 2   example_very_unclear  211225 non-null  bool  
 3   admiration            211225 non-null  int64 
 4   amusement             211225 non-null  int64 
 5   anger                 211225 non-null  int64 
 6   annoyance             211225 non-null  int64 
 7   approval              211225 non-null  int64 
 8   caring                211225 non-null  int64 
 9   confusion             211225 non-null  int64 
 10  curiosity             211225 non-null  int64 
 11  desire                211225 non-null  int64 
 12  disappointment        211225 non-null  int64 
 13  disapproval           211225 non-null  int64 
 14  disgust               211225 non-null  int64 
 15  embarrassment    

In [7]:
df.isnull().sum()

,0
id,0
text,0
example_very_unclear,0
admiration,0
amusement,0
anger,0
annoyance,0
approval,0
caring,0
confusion,0


In [9]:
df.drop(columns=['id','example_very_unclear'], inplace=True)
df.dropna(subset=["text"],inplace=True)
display(df.sample(10))
df.columns

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
126627,Don't worry you'll outlive them.,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83254,15 is too old to play playing fortnite smh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55773,Damn have they not heard of shelters. Bro go a...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92949,"The loving mother referred to her son as ""the ...",0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
95403,Bundle up and sled! Make a snowman! Take a wal...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207876,What is your final solution for Snowflakes?,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
183719,1 And I pulled it back in the same way she did...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
62129,"Sorry mate, you’ve got the wrong sub. This is ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
196918,So lifelike!,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33445,I love him even though the newest season of t@...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


Index(['text', 'admiration', 'amusement', 'anger', 'annoyance', 'approval',
       'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
       'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
       'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride',
       'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')

In [10]:
# text cleaning
def clean_text(text):
  text = str(text).lower()
  text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
  text = re.sub(r'\@w+|\#','', text)
  text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
  text = re.sub(r"\s+", " ", text).strip()
  return text

df["text"] = df["text"].apply(clean_text)

In [11]:
label_cols = df.columns[1:]
labels = df[label_cols].values

In [12]:
labels.shape

(211225, 28)

In [13]:
label_freq = np.sum(labels, axis=0)
print(label_freq)
device = torch.device("cuda" if torch.cuda.is_available()else "cpu")
class_weights = torch.tensor((1/(label_freq + 1e-5)), dtype=torch.float).to(device)
print(f"class weights : {class_weights}")

[17131  9245  8084 13618 17620  5999  7359  9692  3817  8469 11424  5301
  2476  5629  3197 11625   673  7983  8191  1810  8715  1302  8785  1289
  2525  6758  5514 55298]
class weights : tensor([5.8374e-05, 1.0817e-04, 1.2370e-04, 7.3432e-05, 5.6754e-05, 1.6669e-04,
        1.3589e-04, 1.0318e-04, 2.6199e-04, 1.1808e-04, 8.7535e-05, 1.8864e-04,
        4.0388e-04, 1.7765e-04, 3.1279e-04, 8.6022e-05, 1.4859e-03, 1.2527e-04,
        1.2209e-04, 5.5249e-04, 1.1474e-04, 7.6805e-04, 1.1383e-04, 7.7580e-04,
        3.9604e-04, 1.4797e-04, 1.8136e-04, 1.8084e-05], device='cuda:0')


In [17]:
# Tokenization
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
encodings = tokenizer(list(df["text"].values), truncation=True, padding=True,max_length=128)

In [18]:
# Dataset Custom class
class GoEmotionsDataset(Dataset):
  def __init__(self, encodings,labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
     item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
     item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
     return item

  def __len__(self):
    return len(self.labels)



In [ ]:
# Dataset & loader creations

dataset = GoEmotionsDataset(encodings,labels)
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


# train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16)

# Subset creation for faster training


subset_size = 30000
train_size = int(0.8 * subset_size)
val_size = subset_size - train_size

# Create subsets
subset_indices = list(range(subset_size))
train_indices = subset_indices[:train_size]
val_indices = subset_indices[train_size:]

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

#Model Initialization

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels= len(label_cols), problem_type="multi_label_classification"
)

model.to(device)

# using optmizer AdamW with learning rate
optimizer = AdamW(model.parameters(), lr=5e-5)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# 🔁 Training Loop
def train(model, loader):
    model.train()
    total_loss = 0
    for batch in tqdm(loader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# 📏 Evaluation Loop
def evaluate(model, loader):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds.extend(torch.sigmoid(logits).cpu().numpy())
            true.extend(labels.cpu().numpy())
    preds_bin = np.array(preds) >= 0.5
    h_loss = hamming_loss(true, preds_bin)
    f1 = f1_score(true, preds_bin, average='micro')
    return h_loss, f1

In [22]:
# # 🚀 Run Training
# EPOCHS = 3
# num_training_steps = len(train_loader) * EPOCHS

# # LR Scheduler
# lr_scheduler = get_scheduler(
#     "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
# )
# for epoch in range(EPOCHS):
#     print(f"\nEpoch {epoch + 1}/{EPOCHS}")
#     train_loss = train(model, train_loader)
#     h_loss, f1 = evaluate(model, val_loader)
#     print(f"Train Loss: {train_loss:.4f} | Hamming Loss: {h_loss:.4f} | F1 Score: {f1:.4f}")

# # 💾 Save model for Streamlit use
# model.save_pretrained("goemotions_model")
# tokenization.save_pretrained("goemotions_model")

# 🚀 Run Training
EPOCHS = 7
num_training_steps = len(train_loader) * EPOCHS

# LR Scheduler
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    train_loss = train(model, train_loader)
    h_loss, f1 = evaluate(model, val_loader)
    print(f"Train Loss: {train_loss:.4f} | Hamming Loss: {h_loss:.4f} | F1 Score: {f1:.4f}")

# 💾 Save full model for Streamlit use
model.save_pretrained("goemotions_model")
tokenizer.save_pretrained("goemotions_model")

# ✅ Quantize the model (for prediction only)
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Save quantized model
torch.save(quantized_model.state_dict(), "goemotions_model_quantized.pth")

# Save model config separately for loading
with open("goemotions_model/config.json", "w") as f:
    f.write(model.config.to_json_string())



Epoch 1/7


100%|██████████| 1500/1500 [04:06<00:00,  6.10it/s]


Train Loss: 0.1376 | Hamming Loss: 0.0388 | F1 Score: 0.2378

Epoch 2/7


100%|██████████| 1500/1500 [04:08<00:00,  6.05it/s]


Train Loss: 0.1159 | Hamming Loss: 0.0388 | F1 Score: 0.3191

Epoch 3/7


...

Train Loss: 0.0709 | Hamming Loss: 0.0455 | F1 Score: 0.3368

Epoch 6/7


100%|██████████| 1500/1500 [04:08<00:00,  6.04it/s]


Train Loss: 0.0595 | Hamming Loss: 0.0475 | F1 Score: 0.3474

Epoch 7/7


100%|██████████| 1500/1500 [04:08<00:00,  6.04it/s]


Train Loss: 0.0510 | Hamming Loss: 0.0479 | F1 Score: 0.3521


In [ ]:
print(torch.cuda.is_available())



In [23]:
import torch
import numpy as np

def predict_emotions(text, threshold=0.5):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).cpu().numpy()[0]

    predictions = [label for label, prob in zip(label_cols, probs) if prob >= threshold]
    return predictions if predictions else ["Neutral or No strong emotion"]
sample_texts = [
    "I'm feeling really happy and excited today!",
    "This is so frustrating and annoying.",
    "I'm just okay, nothing special.",
    "Why did you do that? I'm so mad at you.",
    "Thank you for your kind help!"
]

for text in sample_texts:
    print(f"\nText: {text}")
    print("Predicted Emotions:", predict_emotions(text, threshold=0.3))



Text: I'm feeling really happy and excited today!
Predicted Emotions: ['excitement', 'joy']

Text: This is so frustrating and annoying.
Predicted Emotions: ['annoyance', 'disgust']

Text: I'm just okay, nothing special.
Predicted Emotions: ['approval', 'relief']

Text: Why did you do that? I'm so mad at you.
Predicted Emotions: ['anger', 'annoyance']

Text: Thank you for your kind help!
Predicted Emotions: ['gratitude']


In [24]:
import shutil

# Zip the goemotions_model folder
shutil.make_archive("goemotions_model", 'zip', "goemotions_model")


'/content/goemotions_model.zip'

In [25]:
from google.colab import files

# Download the zipped model folder
files.download("goemotions_model.zip")

# Download the quantized model separately
files.download("goemotions_model_quantized.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>